# Import packages

In [1]:
import pandas as pd

# Read in data

## Regression

In [2]:
regression_df = pd.read_csv('datasets\cleaned_datasets\BBB_regression.csv')
regression_df

,SMILES,logBB
0,CN1C(=NN=N1)SCC2=C(N3C(C(C3=O)(NC(=O)C(C4=CC=C...,-2.52
1,CN1CC[C@]23[C@@H]4[C@H]1CC5=C2C(=C(C=C5)OC6[C@...,-2.15
2,CN1CC[C@]23[C@@H]4[C@H]1CC5=C2C(=C(C=C5)O)O[C@...,-2.09
3,CC1=NC=C(C=C1)CC2CNC(NC2=O)NCCCCC3=NC=C(C=C3C)Br,-1.88
4,c1(c2c3n(c4c(C(N(C)C3)=O)c(Cl)ccc4)cn2)noc(C(O...,-1.82
...,...,...
1053,C[NH2+]CCCN1C2=CC=CC=C2CCC3=CC=CC=C31,1.20
1054,CN(C)CCCN1C2=CC=CC=C2SC3=CC=CC=C31,1.23
1055,CN(C)CCOC(C1=CC=CC=C1)C2=CC=CC=C2,1.30
1056,CNCCCN1C2=CC=CC=C2SC3=C1C=C(C=C3)Cl,1.40


In [3]:
(regression_df['logBB'] <= -1.01).sum() #These are BBB+

128

In [4]:
(regression_df['logBB'] >= -1).sum() #These are BBB-

930

## Classification

In [5]:
classification_df = pd.read_csv('datasets\cleaned_datasets\BBB_classification.csv')
classification_df

,SMILES,BBB+/BBB-
0,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,BBB-
1,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,BBB-
2,Oc1c(I)cc(Cl)c2cccnc12,BBB-
3,CCNC(=NCCSCc1ncccc1Br)NC#N,BBB-
4,CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,BBB-
...,...,...
7802,c1ccc(CN(CC2=NCCN2)c2ccccc2)cc1,BBB-
7803,CCOCCn1c(N2CCCN(C)CC2)nc2ccccc21,BBB+
7804,CN1CCC(=C2c3ccccc3CC(=O)c3sccc32)CC1,BBB+
7805,Cc1[nH]c(=O)c(C#N)cc1-c1ccncc1,BBB-


In [6]:
classification_df['BBB+/BBB-'].value_counts()

BBB+/BBB-
BBB+    4956
BBB-    2851
Name: count, dtype: int64